# Goal

<h3 style="color:blue">assess the quality of summaries written by students</h3>
<h3 style="color:indigo">evaluate how well a student represents the main idea and details of a source text, as well as the clarity, precision, and fluency of the language used in the summary</h3>
<h3 style="color:red">Freely & publicly available external data is <b>allowed</b>, including pre-trained models</h3>
<h3>This is Multi-Output problem</h3>

### Use Hugging Face Library
### Use NLTK
### Use Tensorflow

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import math
import subprocess
from tqdm import tqdm
import pickle

In [3]:
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, median_absolute_error

In [5]:
import transformers
from transformers import AutoTokenizer, TFBertModel

In [6]:
import keras_tuner 

In [7]:
prompts_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summaries_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

In [8]:
train = pd.merge(prompts_train, summaries_train, on='prompt_id')
test = pd.merge(prompts_test, summaries_test, on='prompt_id')

In [9]:
train.rename(columns = {'text' : 'summary'}, inplace=True)
test.rename(columns = {'text' : 'summary'}, inplace=True)

In [10]:
train.head(2)

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,summary,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058


In [11]:
train['summary'][0]

'1 element of an ideal tragedy is that it should be arranged on a complex plan.  Another element of an ideal tragedy is that it should only have one main issue. The last element of an ideal tragedy is that it should have a double thread plot and an opposite catastrophe for both good and bad.'

In [12]:
columns_needed = ["prompt_text", "summary"]

In [13]:
train_data = train[columns_needed]
test_data = test[columns_needed]

In [14]:
#from transformers import XLNetTokenizer, TFXLNetModel
#tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
#model = TFXLNetModel.from_pretrained('xlnet-base-cased', return_dict=True)

#from transformers import RobertaTokenizer, TFRobertaModel
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base-cased')
#model = TFRobertaModel.from_pretrained('roberta-base-cased', return_dict=True)

from transformers import AutoTokenizer, TFBertModel
model = TFBertModel.from_pretrained('/kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased')

Some layers from the model checkpoint at /kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at /kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### Next time use prepare_tf_dataset which is used to directly tokenize and data colat and
### make dataset compatible with tensorflow
####       https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset

In [15]:

def vectorize_dataframe(dataframe, col):
    vectors = []
    for text in tqdm(dataframe[col].tolist()):
        text_tokens = tokenizer(text, return_tensors="tf",max_length = 512, padding='max_length', truncation=True)
        
        output = model(text_tokens)
        
        pooler_output = output.pooler_output

        vectors.append(pooler_output)
    return vectors
    

In [16]:
test_data['prompt_text_embedded'] = vectorize_dataframe(test_data, 'prompt_text')
test_data['summary_embedded'] = vectorize_dataframe(test_data, 'summary')

100%|██████████| 4/4 [00:00<00:00,  5.44it/s]


In [17]:
with open("/kaggle/input/embeddings/BERT_prompt_text_embeddings.pkl", "rb") as file:
    train_data['prompt_text_embedded'] = pickle.load(file)
    
with open("/kaggle/input/embeddings/BERT_summary_embeddings.pkl", "rb") as file:
    train_data['summary_embedded'] = pickle.load(file)

In [18]:
traning_set = train_data[['prompt_text_embedded', 'summary_embedded']]
testing_set = test_data[['prompt_text_embedded', 'summary_embedded']]

### Take average of embeddings  [Not required, just checking]

In [19]:
target1 = np.array(train['content'])
target1 = target1.astype('float32')

target2 = np.array(train['wording'])
target2 = target2.astype('float32')

#target = (target1, target2)

In [20]:
def convert_tensor_to_numpy(tensor):
        return np.array(tensor, dtype='float32')

traning_set = traning_set.applymap(convert_tensor_to_numpy)
testing_set = testing_set.applymap(convert_tensor_to_numpy)

In [21]:
def prepare_dataset(dataset):
    # Flatten the nested arrays in the DataFrame
    dataset['prompt_text_embedded'] = dataset['prompt_text_embedded'].apply(lambda x: x.flatten())
    dataset['summary_embedded'] = dataset['summary_embedded'].apply(lambda x: x.flatten())
    
    feature1 = np.array(dataset['prompt_text_embedded'].tolist())
    feature2 = np.array(dataset['summary_embedded'].tolist())
    
    features = np.concatenate((feature1, feature2), axis=1)
    
    return features

In [22]:
features = prepare_dataset(traning_set)

In [23]:
features_for_test = prepare_dataset(testing_set)

In [24]:
from tensorflow.keras.layers import Dense, Input, Flatten

In [25]:
def build_model_content(hp):   
   
    #optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    
    model_content = tf.keras.Sequential([
            Dense( units=hp.Int("units1", min_value=252, max_value=356, step=32)),
            Dense( units=hp.Int("units2", min_value=156, max_value=252, step=32)),
            Dense( units=hp.Int("units3", min_value=48, max_value=156, step=32)),
            Dense(1, activation='linear')
    ])
    
    model_content.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae', 'mse'])
    
    return model_content


In [26]:
def build_model_wording(hp):   
   
    #optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    
    model_wording = tf.keras.Sequential([
            Dense( units=hp.Int("units1", min_value=252, max_value=356, step=32)),
            Dense( units=hp.Int("units2", min_value=156, max_value=252, step=32)),
            Dense( units=hp.Int("units3", min_value=48, max_value=156, step=32)),
            Dense(1, activation='linear')
    ])
    
    model_wording.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae', 'mse'])
    
    return model_wording

### hyperband for build_model_content

In [27]:

objective = keras_tuner.Objective('mse', 'min')

content_tuner = keras_tuner.Hyperband(
    hypermodel=build_model_content,
    objective=objective,
    max_epochs=20,
    factor=3,
    hyperband_iterations=3,
    tune_new_entries=True,
    allow_new_entries=True,
    max_retries_per_trial=5,
    max_consecutive_failed_trials=5
)

wording_tuner = keras_tuner.Hyperband(
    hypermodel=build_model_wording,
    objective=objective,
    max_epochs=20,
    factor=3,
    hyperband_iterations=3,
    tune_new_entries=True,
    allow_new_entries=True,
    max_retries_per_trial=5,
    max_consecutive_failed_trials=5
)

In [28]:
content_tuner.search(features, target1,epochs=10, validation_split=0.2)

Trial 81 Complete [00h 00m 08s]
mse: 0.36659523844718933

Best mse So Far: 0.3224892020225525
Total elapsed time: 00h 12m 35s


In [29]:
wording_tuner.search(features, target2,epochs=10, validation_split=0.2)

Trial 79 Complete [00h 00m 11s]
mse: 0.5533625483512878

Best mse So Far: 0.5006468892097473
Total elapsed time: 00h 12m 21s


In [30]:
# Get the optimal hyperparameters
best_content_tuner_hps=content_tuner.get_best_hyperparameters(num_trials=1)[0]

# Get the optimal hyperparameters
best_wording_tuner_hps=wording_tuner.get_best_hyperparameters(num_trials=1)[0]

In [31]:
best_content_tuner_hps.values

{'units1': 284,
 'units2': 220,
 'units3': 48,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0013'}

In [32]:
best_wording_tuner_hps.values

{'units1': 316,
 'units2': 156,
 'units3': 112,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0013'}

In [40]:
content_hp_model = content_tuner.hypermodel.build(best_content_tuner_hps)
history__1 = content_hp_model.fit(features, target1, epochs=100)

Epoch 1/100
224/224 [==============================] - 2s 3ms/step - loss: 0.6036 - mae: 0.5904 - mse: 0.6036
Epoch 2/100
224/224 [==============================] - 1s 3ms/step - loss: 0.4253 - mae: 0.5060 - mse: 0.4253
Epoch 3/100
224/224 [==============================] - 1s 4ms/step - loss: 0.4049 - mae: 0.4922 - mse: 0.4049
Epoch 4/100
224/224 [==============================] - 1s 3ms/step - loss: 0.3987 - mae: 0.4905 - mse: 0.3987
Epoch 5/100
224/224 [==============================] - 1s 3ms/step - loss: 0.3883 - mae: 0.4854 - mse: 0.3883
Epoch 6/100
224/224 [==============================] - 1s 3ms/step - loss: 0.3723 - mae: 0.4757 - mse: 0.3723
Epoch 7/100
224/224 [==============================] - 1s 3ms/step - loss: 0.3665 - mae: 0.4713 - mse: 0.3665
Epoch 8/100
224/224 [==============================] - 1s 4ms/step - loss: 0.3573 - mae: 0.4631 - mse: 0.3573
Epoch 9/100
224/224 [==============================] - 1s 3ms/step - loss: 0.3595 - mae: 0.4633 - mse: 0.3595
Epoch 10/1

In [41]:
wording_hp_model = wording_tuner.hypermodel.build(best_wording_tuner_hps)
history__2 = wording_hp_model.fit(features, target2, epochs=150)

Epoch 1/150
224/224 [==============================] - 2s 4ms/step - loss: 0.7180 - mae: 0.6642 - mse: 0.7180
Epoch 2/150
224/224 [==============================] - 1s 4ms/step - loss: 0.6199 - mae: 0.6184 - mse: 0.6199
Epoch 3/150
224/224 [==============================] - 1s 4ms/step - loss: 0.5984 - mae: 0.6048 - mse: 0.5984
Epoch 4/150
224/224 [==============================] - 1s 3ms/step - loss: 0.5519 - mae: 0.5802 - mse: 0.5519
Epoch 5/150
224/224 [==============================] - 1s 3ms/step - loss: 0.5457 - mae: 0.5789 - mse: 0.5457
Epoch 6/150
224/224 [==============================] - 1s 3ms/step - loss: 0.5460 - mae: 0.5778 - mse: 0.5460
Epoch 7/150
224/224 [==============================] - 1s 3ms/step - loss: 0.5403 - mae: 0.5752 - mse: 0.5403
Epoch 8/150
224/224 [==============================] - 1s 3ms/step - loss: 0.5227 - mae: 0.5653 - mse: 0.5227
Epoch 9/150
224/224 [==============================] - 1s 3ms/step - loss: 0.5138 - mae: 0.5605 - mse: 0.5138
Epoch 10/1

In [42]:
evaluate_on_train_content = content_hp_model.evaluate(features, target1)
evaluate_on_train_wording = wording_hp_model.evaluate(features, target2)

224/224 [==============================] - 1s 3ms/step - loss: 0.4140 - mae: 0.5070 - mse: 0.4140


In [43]:
print('evaluate_on_train_content', evaluate_on_train_content)
print('evaluate_on_train_wording',evaluate_on_train_wording)

evaluate_on_train_content [0.28349781036376953, 0.4145551323890686, 0.28349781036376953]
evaluate_on_train_wording [0.41404685378074646, 0.5070337653160095, 0.41404685378074646]


In [44]:
content_prediction = content_hp_model.predict(features)
wording_prediction = wording_hp_model.predict(features)

224/224 [==============================] - 0s 2ms/step


### Predict on test

In [45]:
test_pred_content = content_hp_model.predict(features_for_test)
test_pred_wording = wording_hp_model.predict(features_for_test)

1/1 [==============================] - 0s 19ms/step


## submission

In [46]:
test_pred_content = test_pred_content.reshape(-1)
test_pred_wording = test_pred_wording.reshape(-1)

In [47]:
submission = pd.DataFrame({
    'student_id' : test['student_id'],
    'content' : test_pred_content,
    'wording' : test_pred_wording
})

In [48]:
submission.to_csv('submission.csv', index=False)

In [49]:
submission.head()

,student_id,content,wording
0,000000ffffff,-1.533198,-1.301491
1,222222cccccc,-1.342265,-1.364608
2,111111eeeeee,-1.517045,-1.435132
3,333333dddddd,-1.427587,-1.395533
